In [1]:
import requests
import json
import re
from datetime import datetime as dt
from dateutil.parser import parse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
username = 'XXX'
password = 'XXX'
Server = "http://wsc.aquaticinformatics.net/AQUARIUS/publish/v2/"
server_pro = "https://wsc.aquaticinformatics.net/AQUARIUS/Provisioning/v1/"
# list of stations,
station_list = ['05BB001','05HG001']; # can be loaded from a file

In [3]:
def generateRatingCurve(stationIDs, token):
    # for each station
    RC_df = pd.DataFrame() # initializing the date frame that holds the information of rating curves
    RC_df_temp = pd.DataFrame() # initializing the temporary data frame
    # populating the temporal data frame
    RC_df_temp = RC_df_temp.append({"Rating_Id": 1,\
                                    "Type":1,\
                                    "Remarks":1,\
                                    "InputValue":1,\
                                    "OutputValue":1,\
                                    "StartTime":1,\
                                    "EndTime":1,\
                                    "Station_Id":1,\
                                    "locationName":1},ignore_index=True)
    
    for station in stationIDs:
        # load the rating curve infromation
        print (station)
        
        try:
            req = requests.get(Server + 'GetRatingCurveList?RatingModelIdentifier='+\
                               'Stage-Discharge.Rating Curve@' + station + '&token=' + token)
            RatingCurves = req.json()['RatingCurves']
        except:
            print ("Failed to load Field Visit Data for Station:" + station)
            continue
            
        try:
            req = requests.get(Server + 'GetLocationData?LocationIdentifier=' + station + '&token=' + token)
            locationName = req.json()['LocationName']
        except:
            print ("Failed to get station name for station:" + station)
            continue

        ## reading the data
        for i in range(len(RatingCurves)): # loop over number of rating curve available
            # get the entire information of a Rating Curve Id
            RatingCurve = RatingCurves[i] # get the rating curve information
            # get the general information of the the rating curve
            RatingCurveId = RatingCurve['Id'] # save the rating curve number
            RatingCurveType = RatingCurve['Type'] # type of the rating curve, Linear, Logarithmic
            RatingCurveRemarks = RatingCurve['Remarks'] # if any remarks exists
            RatingCurvePeriodsOfApplicability = RatingCurve ['PeriodsOfApplicability'][0]
            RatingCurvePeriodsOfApplicabilityStartTime = RatingCurvePeriodsOfApplicability['StartTime']
            RatingCurvePeriodsOfApplicabilityEndTime = RatingCurvePeriodsOfApplicability['EndTime']
            RatingCurveBaseRatingTable = RatingCurve['BaseRatingTable'] # get the Base Rating Table
            
            for j in range(len(RatingCurveBaseRatingTable)): # loop over the numner of point in the rating curve
                point = RatingCurveBaseRatingTable[j] # get the information of the point
                # populating the data frame
                RC_df_temp ['Rating_Id'] = RatingCurveId # rating curve Id
                RC_df_temp ['Type'] = RatingCurveType # Rating Curve type, Linear or logarithmic
                RC_df_temp ['Remarks'] = RatingCurveRemarks # Remarks
                RC_df_temp ['InputValue'] = point['InputValue'] # elevation
                RC_df_temp ['OutputValue'] = point['OutputValue'] # discharge
                RC_df_temp ['StartTime'] = RatingCurvePeriodsOfApplicabilityStartTime #start of rating curve
                RC_df_temp ['EndTime'] = RatingCurvePeriodsOfApplicabilityEndTime # end of rating curve
                RC_df_temp ['Station_Id'] = station # name of the station
                RC_df_temp ['locationName'] = locationName # name of the station
                # appending to the general dataframe
                RC_df = RC_df.append(RC_df_temp) # populate the infromation of the rating curve point
                
        print(RC_df) # it can be saved
        
def main():
    stationIDs = station_list
    token = getToken(username, password)
    generateRatingCurve(stationIDs, token)
    
def getToken(userName, password):
    s = requests.Session()
    data = '{"Username": "' + userName + '", "EncryptedPassword": "' + password + '", "Locale": ""}'
    url = server_pro + 'session'
    s.get(url)
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    r = s.post(url, data=data, headers=headers)
    token = r.text
    return token

if __name__ == "__main__":
    main()

05BB001
{'Id': '14.000', 'Type': 'LinearTable', 'Remarks': '', 'InputParameter': {'ParameterName': 'Stage', 'ParameterUnit': 'm'}, 'OutputParameter': {'ParameterName': 'Discharge', 'ParameterUnit': 'm^3/s'}, 'PeriodsOfApplicability': [{'StartTime': '1996-01-01T00:00:00.0000000-07:00', 'EndTime': '1999-01-01T00:00:00.0000000-07:00', 'Remarks': ''}], 'Shifts': [{'PeriodOfApplicability': {'StartTime': '1996-01-11T11:10:00.0000000-07:00', 'EndTime': '9999-12-31T23:59:59.9999999+00:00', 'Remarks': 'MMNT, CIC                           '}, 'ShiftPoints': [{'InputValue': 0, 'Shift': -0.229}]}, {'PeriodOfApplicability': {'StartTime': '1996-01-29T11:50:00.0000000-07:00', 'EndTime': '9999-12-31T23:59:59.9999999+00:00', 'Remarks': 'MMNT, CIC                           '}, 'ShiftPoints': [{'InputValue': 0, 'Shift': -0.11}]}, {'PeriodOfApplicability': {'StartTime': '1996-02-21T11:00:00.0000000-07:00', 'EndTime': '9999-12-31T23:59:59.9999999+00:00', 'Remarks': 'MMNT, CIC WITH OPEN LEADS           '}, 

{'Id': '18.000', 'Type': 'LogarithmicTable', 'Remarks': 'Curve used to produce data however it was never approved.  Approved to close off the rating period manager and move to Curve 19.000.', 'InputParameter': {'ParameterName': 'Stage', 'ParameterUnit': 'm'}, 'OutputParameter': {'ParameterName': 'Discharge', 'ParameterUnit': 'm^3/s'}, 'PeriodsOfApplicability': [{'StartTime': '2013-01-01T00:00:00.0000000-07:00', 'EndTime': '2015-01-01T00:00:00.0000000-07:00', 'Remarks': 'New curve drawn for 2013'}], 'Shifts': [{'PeriodOfApplicability': {'StartTime': '2013-01-01T00:00:01.0000000-07:00', 'EndTime': '9999-12-31T23:59:59.9999999+00:00', 'Remarks': 'new curve 18 starts; shift from Jan 9 mnt'}, 'ShiftPoints': [{'InputValue': 0.1, 'Shift': -0.05}]}, {'PeriodOfApplicability': {'StartTime': '2013-01-09T13:09:00.0000000-07:00', 'EndTime': '9999-12-31T23:59:59.9999999+00:00', 'Remarks': 'mmnt Q=11.3 ; mostly ice cover at gauge, large open area at control'}, 'ShiftPoints': [{'InputValue': 0.1, 'Shi

{'Id': '12.000', 'Type': 'LinearTable', 'Remarks': '', 'InputParameter': {'ParameterName': 'Stage', 'ParameterUnit': 'm'}, 'OutputParameter': {'ParameterName': 'Discharge', 'ParameterUnit': 'm^3/s'}, 'PeriodsOfApplicability': [{'StartTime': '1987-01-01T00:00:00.0000000-06:00', 'EndTime': '1995-01-01T00:00:00.0000000-06:00', 'Remarks': ''}], 'Shifts': [], 'BaseRatingTable': [{'InputValue': 1.93, 'OutputValue': 18.5}, {'InputValue': 1.94, 'OutputValue': 20.9}, {'InputValue': 1.95, 'OutputValue': 23.5}, {'InputValue': 1.96, 'OutputValue': 26.3}, {'InputValue': 1.97, 'OutputValue': 29.3}, {'InputValue': 1.98, 'OutputValue': 32.6}, {'InputValue': 1.99, 'OutputValue': 36.1}, {'InputValue': 2, 'OutputValue': 39.8}, {'InputValue': 2.01, 'OutputValue': 43.7}, {'InputValue': 2.02, 'OutputValue': 47.8}, {'InputValue': 2.03, 'OutputValue': 52}, {'InputValue': 2.04, 'OutputValue': 56.4}, {'InputValue': 2.05, 'OutputValue': 60.9}, {'InputValue': 2.06, 'OutputValue': 65.5}, {'InputValue': 2.07, 'Outp

{'Id': '12.010-SC1', 'Type': 'LinearTable', 'Remarks': '', 'InputParameter': {'ParameterName': 'Stage', 'ParameterUnit': 'm'}, 'OutputParameter': {'ParameterName': 'Discharge', 'ParameterUnit': 'm^3/s'}, 'PeriodsOfApplicability': [{'StartTime': '2007-01-01T00:00:01.0000000-06:00', 'EndTime': '2010-01-01T00:00:01.0000000-06:00', 'Remarks': ''}], 'Shifts': [], 'BaseRatingTable': [{'InputValue': 1.93, 'OutputValue': 18.5}, {'InputValue': 1.94, 'OutputValue': 20.9}, {'InputValue': 1.95, 'OutputValue': 23.5}, {'InputValue': 1.96, 'OutputValue': 26.3}, {'InputValue': 1.97, 'OutputValue': 29.3}, {'InputValue': 1.98, 'OutputValue': 32.6}, {'InputValue': 1.99, 'OutputValue': 36.1}, {'InputValue': 2, 'OutputValue': 39.8}, {'InputValue': 2.01074, 'OutputValue': 43.7}, {'InputValue': 2.02151, 'OutputValue': 47.8}, {'InputValue': 2.03231, 'OutputValue': 52}, {'InputValue': 2.04314, 'OutputValue': 56.4}, {'InputValue': 2.05399, 'OutputValue': 60.9}, {'InputValue': 2.06487, 'OutputValue': 65.5}, {'In

{'Id': '15.000', 'Type': 'LogarithmicTable', 'Remarks': 'Curve Re-drawn ', 'InputParameter': {'ParameterName': 'Stage', 'ParameterUnit': 'm'}, 'OutputParameter': {'ParameterName': 'Discharge', 'ParameterUnit': 'm^3/s'}, 'PeriodsOfApplicability': [{'StartTime': '2013-01-01T00:00:01.0000000-06:00', 'EndTime': '2014-01-01T00:00:00.0000000-06:00', 'Remarks': 'For use in 2014'}, {'StartTime': '2014-01-01T00:00:00.0000000-06:00', 'EndTime': '2014-05-31T23:59:59.0000000-06:00', 'Remarks': 'For use in 2014'}, {'StartTime': '2014-05-31T23:59:59.0000000-06:00', 'EndTime': '2015-01-01T00:00:00.0000000-06:00', 'Remarks': 'In use for the remainder of 2014'}, {'StartTime': '2015-01-01T00:00:00.0000000-06:00', 'EndTime': '2016-01-01T00:00:00.0000000-06:00', 'Remarks': ''}, {'StartTime': '2016-01-01T00:00:00.0000000-06:00', 'EndTime': '2016-09-16T00:00:00.0000000-06:00', 'Remarks': ''}, {'StartTime': '2016-09-16T00:00:00.0000000-06:00', 'EndTime': '2017-01-11T12:05:00.0000000-06:00', 'Remarks': 'APPRO